# Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [83]:
from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import cross_val_score

# Raw data 

In [2]:
data = pd.read_csv("data/train.csv")

**Nous avons 30 051 patients avec 16 features et une target**

In [5]:
data

,USMER,MEDICAL_UNIT,SEX,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,LABEL
0,2,12,1,2,21,2,2,2,2,2,2,2,2,2,2,2,SOFT_COVID
1,2,4,1,2,49,2,2,2,2,2,2,2,2,2,2,2,SOFT_COVID
2,2,9,2,1,60,2,2,2,2,2,2,2,2,2,1,2,DEAD
3,2,12,1,1,55,2,2,2,2,2,2,2,2,1,2,2,DEAD
4,2,12,1,2,47,2,2,2,2,2,2,2,2,2,2,2,SOFT_COVID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30046,2,12,1,2,40,2,2,2,2,2,2,2,2,1,2,2,SOFT_COVID
30047,1,4,2,2,76,2,1,2,2,2,1,2,1,2,1,2,DEAD
30048,2,9,1,2,34,2,2,2,1,2,2,2,2,2,2,2,SOFT_COVID
30049,1,4,1,2,60,2,1,2,1,2,1,2,2,2,2,2,DEAD


In [4]:
data.describe()

,USMER,MEDICAL_UNIT,SEX,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO
count,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.000000,30051.00000
mean,1.536787,8.145619,1.589664,1.485441,50.205417,1.992679,1.748128,1.968254,1.976074,1.971914,1.713321,1.950950,1.956940,1.792819,1.956041,1.91937
std,0.498653,3.739668,0.491903,0.499796,19.519962,0.085250,0.434095,0.175326,0.152821,0.165220,0.452218,0.215976,0.202996,0.405293,0.205006,0.27227
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
25%,1.000000,4.000000,1.000000,1.000000,38.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.00000
50%,2.000000,9.000000,2.000000,1.000000,52.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000
75%,2.000000,12.000000,2.000000,2.000000,64.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000
max,2.000000,13.000000,2.000000,2.000000,104.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000


## Data analysis

## Features analysis

USMER : indique si le patient a traité des unités médicales de premier ou de deuxième niveau

In [7]:
data["USMER"].value_counts()

2    16131
1    13920
Name: USMER, dtype: int64

MEDICAL_UNIT : type d'institution du système national de santé qui a dispensé les soins (de 1 à 13)

SEX : sexe du patient (1 pour les femmes et 2 pour les hommes)

PNEUMONIA : si le patient a déjà une inflammation des sacs alvéolaires des poumons (1) ou non (2)

AGE : âge du patient

PREGNANT : si la patiente est enceinte (1) ou non (2)

DIABETES : si le patient est diabétique (1) ou non (2)

COPD : si le patient est atteint de bronchopneumopathie chronique obstructive(1) ou non (2)

ASTHMA : si le patient est asthmatique (1) ou non (2)

INMSUPR : si le patient est immunodéprimé (1) ou non (2)

HIPERTENSION : si le patient est hypertendu (1) ou non (2)

OTHER_DISEASE : si le patient est atteint d'une autre maladie (1) ou non (2)

CARDIOVASCULAR : si le patient est atteint d'une maladie du cœur ou des vaisseaux sanguins (1) ou non (2)

OBESITY : si le patient est obèse (1) ou non (2)

RENAL_CHRONIC : si le patient est atteint d'une maladie rénale chronique (1) ou non (2)

TOBACCO : si le patient fume (1) ou non (2)

**Except for age, all other features are categorical** 

## Target analysis

In [18]:
data["LABEL"].value_counts()

SOFT_COVID      10017
DEAD            10017
STRONG_COVID    10017
Name: LABEL, dtype: int64

**The dataset is balanced**

## Missing data

In [17]:
data.isna().sum()

USMER             0
MEDICAL_UNIT      0
SEX               0
PNEUMONIA         0
AGE               0
PREGNANT          0
DIABETES          0
COPD              0
ASTHMA            0
INMSUPR           0
HIPERTENSION      0
OTHER_DISEASE     0
CARDIOVASCULAR    0
OBESITY           0
RENAL_CHRONIC     0
TOBACCO           0
LABEL             0
dtype: int64

**There is no missing data in the dataset**

# Pipeline

In [20]:
X = data.drop(columns="LABEL")

In [93]:
y=data["LABEL"]

y = y.map({"SOFT_COVID" : 0,
          "STRONG_COVID" : 1,
          "DEAD" : 2})

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
ohe_encoder = OneHotEncoder(drop="if_binary",sparse='False')

In [26]:
# Preprocessing pipe
num_transformer = Pipeline([ 
    ('scaling', StandardScaler())
     ])

cat_transformer = Pipeline([
    ('encoder',ohe_encoder)
])

In [71]:
num_col = ["AGE"]
cat_col = X.columns.tolist()

In [72]:
cat_col.remove("AGE")
cat_col

['USMER',
 'MEDICAL_UNIT',
 'SEX',
 'PNEUMONIA',
 'PREGNANT',
 'DIABETES',
 'COPD',
 'ASTHMA',
 'INMSUPR',
 'HIPERTENSION',
 'OTHER_DISEASE',
 'CARDIOVASCULAR',
 'OBESITY',
 'RENAL_CHRONIC',
 'TOBACCO']

In [73]:
preproc_baseline = ColumnTransformer([
    ('num_tr', num_transformer, num_col),
    ('cat_tr', cat_transformer, cat_col)],
    remainder='drop'
)

In [75]:
X_transformed = preproc_baseline.fit_transform(X)

# Model

In [81]:
from sklearn.metrics import accuracy_score

## Decision Tree Regressor

In [99]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()

In [100]:
pipe_baseline = make_pipeline(preproc_baseline,tree)
pipe_baseline


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_tr',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  ['AGE']),
                                                 ('cat_tr',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse='False'))]),
                                                  ['USMER', 'MEDICAL_UNIT',
                                                   'SEX', 'PNEUMONIA',
                                                   'PREGNANT', 'DIABETES',
                                                   'COPD', 'ASTHMA', 'INMSUPR',
                                                   'HIPERTENSION',
                                                   'OTHER_DISEASE',
                                                   'CARDIOVASCULAR', 'OBESITY',
                                                   'RENAL_CHRONIC',
                                                   'TOBACCO'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [102]:
score_baseline = cross_val_score(pipe_baseline, X, y, cv=5).mean()

In [103]:
score_baseline

0.5984160708141564

# Random Forest

In [111]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


random_forest=  RandomForestClassifier()

pipe_random_forest = make_pipeline(preproc_baseline,random_forest)
pipe_random_forest

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_tr',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  ['AGE']),
                                                 ('cat_tr',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse='False'))]),
                                                  ['USMER', 'MEDICAL_UNIT',
                                                   'SEX', 'PNEUMONIA',
                                                   'PREGNANT', 'DIABETES',
                                                   'COPD', 'ASTHMA', 'INMSUPR',
                                                   'HIPERTENSION',
                                                   'OTHER_DISEASE',
                                                   'CARDIOVASCULAR', 'OBESITY',
                                                   'RENAL_CHRONIC',
                                                   'TOBACCO'])])),
                ('randomforestclassifier', RandomForestClassifier())])

In [113]:
score_random_forest = cross_val_score(pipe_random_forest, X, y, cv=5).mean()

In [114]:
score_random_forest

0.6220424950264505